In [151]:
import ROOT
import math
from ROOT import TH1F, TF1, TColor

rndmGenerator = ROOT.TRandom3()

def bhathat(mu, s, tau, n, m):
    return (n+m-(1+tau)*mu*s)/(2+2*tau)+(((n+m-(1+tau)*mu*s)**2+(1+tau)*m*mu*s)/(4*(1+tau)**2))**0.5
def factorial(n):
    n_factorial = 1
    if n < 0:
        print("error")
    elif n == 0:
        n_factorial = 1
    else: 
        for i in range(1,n + 1):
            n_factorial = n_factorial*i
    return n_factorial

def likelihood(mu, s, b, tau, n, m):
    return (mu*s+b)**n*math.exp(-(mu*s+b))*(tau*b)**m*math.exp(-tau*b)/(factorial(n)*factorial(m))

def qmu(mu, s, b, tau):
    n=rndmGenerator.Poisson(mu*s+b)
    m=rndmGenerator.Poisson(tau*b)
    mu_hat = (n-m/tau)/s
    b_hat = m/tau
    b_hat_hat = bhathat(mu, s, tau, n, m)
    if mu_hat < mu:
        likelihood_mu_bhat = likelihood(mu, s, b_hat_hat, tau, n, m)
        likelihood_muhat_bhathat = likelihood(mu_hat, s, b_hat, tau, n, m)
        likelihoodRatio_mu = likelihood_mu_bhat/likelihood_muhat_bhathat
        q_mu = -2*math.log(likelihoodRatio_mu)
    else:
        q_mu = 0
    return q_mu

# print ("2:", factorial(4) )
# print ("e : ", log_likelihood(0,2,2,1,2,2) )

In [152]:
%jsroot on

In [153]:
def func_fq0(q0):
    if q0[0] > 1.e-10:
        fq = 0.5*(2*math.pi)**(-0.5)*(q0[0])**(-0.5)*math.exp(-0.5*q0[0])
    else:
        fq = 1
    return fq

fq0 = TF1("fq0",func_fq0,0,40)
# c_f = ROOT.TCanvas()
# c_f.SetLogy()
# fq0.Draw()
# c_f.Draw()

In [173]:
h_b2 = ROOT.TH1F("h_b2","f(q0|0);q_{0};f(q_{0}|0)",20, 0, 40)
h_b5 = ROOT.TH1F("h_b5","f(q0|0);q_{0};f(q_{0}|0)",20, 0, 40)
h_b20 = ROOT.TH1F("h_b20","f(q0|0);q_{0};f(q_{0}|0)",20, 0, 40)

mu=0
s=10
b2=2
b5=5
b20=20
tau=1
for i in range(10000000):
    h_b2.Fill(qmu(mu, 10, b2, tau))
    h_b5.Fill(qmu(mu, 10, b5, tau))
    h_b20.Fill(qmu(mu, 10, b20, tau))

Warning in <TROOT::Append>: Replacing existing TH1: h_b2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_b5 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_b20 (Potential memory leak).


In [174]:
c = ROOT.TCanvas()
c.SetLogy()
h_b2.Scale(1./h_b2.Integral())
h_b5.Scale(1./h_b5.Integral())
h_b20.Scale(1./h_b20.Integral())
h_b2.SetMaximum(10)
h_b2.SetMinimum(10e-9)
h_b2.SetLineColor(2) # red
h_b2.SetLineWidth(2)
h_b5.SetLineColor(44) # brown
h_b5.SetLineWidth(3)
h_b5.SetLineStyle(3)
h_b20.SetLineColor(4) # blue
h_b20.SetLineWidth(3)
h_b20.SetLineStyle(3)
h_b2.Draw("hist")
h_b5.Draw("hist same")
h_b20.Draw("hist same")
fq0.SetLineColor(1)
fq0.Draw("same")
c.Draw()